#Importing packages

In [13]:
import pandas as pd 
import cv2
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from matplotlib import patches

#import h5py    
import numpy as np  
import pandas as pd

from scipy.optimize import linear_sum_assignment
#! pip install mpmath
from mpmath import *
import datetime as dt
import json 
import copy

import os 
import time 
import plotly.express as px
import plotly.graph_objects as go
import motmetrics as mm
import numpy as np
import re
os.chdir("../evaluation")
!pwd

/home/sophie/uncertain-identity-aware-tracking/evaluation


#Util functions

In [14]:

def find_rectangle (pts,image_file=None):
    """
    this function provide straigth rectangle of a  set of points
    """
    pts = pts[~np.isnan(pts).any(axis=1)]
    pts= pts.astype(int)
    #print(pts)

    x,y,w,h = cv2.boundingRect(pts)
    if image_file!=None:
        try:
            image = mpimg.imread(image_file)
            fig, ax = plt.subplots(1,figsize=(20,15))
            rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none') # Coordinates: (left, top), width, height
            ax.add_patch(rect)
            ax.scatter(pts[:, 0],pts[:,1], marker="o", color="red", s=2)
            ax.scatter(x+w/2,y+h/2, marker="o", color="yellow", s=2)
            ax.imshow(image)
            plt.show()
        except:
            print("Une erreur durant l'affichage du rectange droit minimal")

    return (x,y,w,h)

#Test the find_rectangle fonction
"""pts=np.array([[785,43],[798,78],[827,48],[833,69],[947,132]])
image_file="pig_tracking_dataset/labeled-data/GR77_20200512_111314/img00000.png"
find_rectangle(pts,image_file=image_file)"""

#keep data  in dictionnary of image of individual of body part: function dlc_data_to_dct
#labels=pd.read_csv("pig_tracking_dataset/labeled-data/GR77_20200512_111314/CollectedData_CDPQ_experiment.csv")






def read_data(file):
    #here we will go through detections of deepsort 
    import json
    track={}
    with open(file) as f:
        json_file = json.load(f) 

    for frame, detections in json_file.items():
        frame=int(frame)
        track[frame]={}
        for id, detection in detections.items():
            if int(id)>4800:  ###We add this condition to stay on our setting where only real identities are considered
                track[frame][id]={}
                track[frame][id]["rectangle"]= tuple(detection)

    return track

    

def read_data(file):
    #here we will go through detections of deepsort 
    import json
    track={}
    with open(file) as f:
        json_file = json.load(f) 

    for frame, detections in json_file.items():
        frame=int(frame)
        track[frame]={}
        for id, detection in detections.items():
            track[frame][id]={}
            track[frame][id]["rectangle"]= tuple(detection)

    return track



def iou (boxA,boxB):
    boxA=[boxA[0],boxA[1],boxA[0]+boxA[2],boxA[1]+boxA[3]]
    boxB=[boxB[0],boxA[1],boxB[0]+boxB[2],boxB[1]+boxB[3]]
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1))
    boxBArea = abs((boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1))
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou  #np.linalg.norm(np.array([float(track[0]), float(track[1])+float(track[3])/2])-np.array([600,17.5]))



"""
Paramètres de l'évaluation:
- le max_iou: fiixé à 0.85 soit on accepte les  iou de 15%
"""
################################################################################################
def evaluate (tracks,track_result, limit=float("inf")) :
    acc = mm.MOTAccumulator(auto_id=False)
    for key, track in tracks.items():
        if key in  track_result.keys()  :
            frame_detection=track_result[key]
            real_animals=np.array([list(individual["rectangle"]) for individual  in list(track.values())])
            hypothesis_animal= np.array([list(individual["rectangle"]) for individual  in list(frame_detection.values())])
            #print(real_animals, "*****", hypothesis_animal)
            distances = mm.distances.iou_matrix(real_animals, hypothesis_animal, max_iou=0.75)
            #print("deepsort",hypothesis_animal)
            #print("real",real_animals)
            #print(distances)
            #print(real_animals)
            acc.update([float(key1) for key1 in list(track.keys())],[ float(re.search(r'^\d+', key1)[0]) for key1 in frame_detection.keys()],distances, frameid=int(key))
        if int(key)>= limit:
              break
    return acc 


#acc_deepsort= evaluate(tracks,deepsort_track)
#acc_deeplabcut=evaluate(tracks, deeplabcut_track)
def Evaluate_over_time(tracks, deepsort_track,column_indice): 
    accs_deepsort=[]
    frames=[]
    prev=0
    for key, track in tracks.items():
        if key-prev>99:
            acc=evaluate(tracks,deepsort_track,limit=int(key))
            accs_deepsort.append(acc)
            frames.append(int(key))
            prev=int(key)
            
    mh = mm.metrics.create()
    summary = mh.compute_many(
        accs_deepsort,
        #[acc_deepsort, acc_deeplabcut],
        #metrics=mm.metrics.motchallenge_metrics,
        #metrics=['idf1','num_frames', 'mota', 'motp'],
        names=frames, #['deepsort','deeplabcut'],
        generate_overall=True
        )

    strsummary = mm.io.render_summary(
        summary,
        formatters=mh.formatters,
        namemap=mm.io.motchallenge_metric_names
    )
    print("cumulative summary")
    print(strsummary)
    for col in summary.columns:
        summary.columns = summary.columns.str.replace(col, col+'_'+column_indice)
    return summary

def Evaluate_single_frames_over_time(tracks, deepsort_track,column_indice): 
    ""
    accs_deepsort=[]
    frames=[]
    for key, track in tracks.items():
        #############################################to arrange for other datasets
        acc=evaluate({ 16:tracks[16], 0:tracks[0],key:tracks[key]},deepsort_track,limit=int(2))
        accs_deepsort.append(acc)
        frames.append(int(key))
        
    mh = mm.metrics.create()
    summary = mh.compute_many(
        accs_deepsort,
        #[acc_deepsort, acc_deeplabcut],
        #metrics=mm.metrics.motchallenge_metrics,
        #metrics=['idf1','num_frames', 'mota', 'motp'],
        names=frames, #['deepsort','deeplabcut'],
        generate_overall=True)
    strsummary = mm.io.render_summary(
        summary,
        formatters=mh.formatters,
        namemap=mm.io.motchallenge_metric_names)
    print("summary per frame")
    #print(summary)
    print(strsummary)
    for col in summary.columns:
        summary.columns = summary.columns.str.replace(col, col+'_'+column_indice)
    
    return summary





def precise_accuracy_track(label_track, model_track, basic_tracker=False):
    """cette fonction calcule le f1-score recall, accuracy des model par rapport au background
    dedans, les score des trackers et des hMM based tracker sont different car le hmmbased tracker a les vrais identités donc facilite les calculs 

    il y'a des id de track qu'il ne retourne pas dans son fichier de resultat.

    Args:
        label_track (_type_): _description_
        model_track (_type_): _description_
        basic_tracker (bool, optional): _description_. Defaults to False.

    Returns:
        _type_: _description_
    """
    import re
    start=True 
    nbr_frame=0
    acc =0
    rec=0
    accs=[]
    f1s=[]
    recs=[]
    indices=[]
    
    accs_ind=[]
    f1s_ind=[]
    recs_ind=[]
    """if basic_tracker==True:
        matching={}
        for frame_id in label_track.keys():
            for label_atq, label_box in label_track[frame_id].items() : 
                if label_atq not in matching.keys() and label_atq!="observed":
                    matching[label_atq]=None 
                    break"""
            
    
    def match_track_and_atq(label_track, model_track):
            """
            retourne au format label_key:model_key les correspondances entre les identités du model et nos label et ceci pour chaque label identity
            """
            matching={}
            for frame_id in label_track.keys():
                if frame_id in model_track.keys() :
                    
                    for label_atq, label_box in label_track[frame_id].items() : 
                        if label_atq not in matching.keys():
                            max_iou=float('-inf')
                            for model_atq, model_box in model_track[frame_id].items(): 
                                #print(model_atq)
                                if  label_atq!="observed" and model_atq!="observed":#fix the problem with the obseved on the label 
                                    
                                    """if str(model_atq)==str(label_atq):
                                        print(model_atq, model_box["rectangle"], label_box["rectangle"])
                                    """
                                    
                                    tmp = iou(model_box["rectangle"], label_box["rectangle"])
                                    if tmp>=max_iou:
                                        matching [label_atq]=model_atq
                                        max_iou =tmp
                
                if len(matching.keys())== len(label_track[frame_id].keys()):
                    break
            return {value:key for key, value in matching.items() if float(key)>4800} #why if float(key)>4800? We have this specific condition because we want to access the basic tracker on the same base with the tracker so only on identities the tracker also have
    
    if basic_tracker==True:
        matching = match_track_and_atq(label_track, model_track)
        print(matching)
    
    for frame_id in label_track.keys() :
        if frame_id in model_track.keys() :
            nbr_frame+=1
            if frame_id==10066:
                print("stop")
            matching_frame={}
            for model_atq, model_box in model_track[frame_id].items() :
                    max_iou=float('-inf')
                    for  label_atq, label_box in label_track[frame_id].items():
                        if  label_atq!="observed" and model_atq!="observed" :#fix the problem with the obseved on the label 
                            tmp = iou(model_box["rectangle"], label_box["rectangle"])
                            if tmp>max_iou:
                                if basic_tracker==True:
                                    if model_atq in matching.keys(): #ca c'est pour les modèles qui crèent trop de nouvelles identités
                                        matching_frame[matching[model_atq] ]=label_atq 
                                else:
                                    matching_frame[model_atq ]=label_atq 
                                max_iou =tmp
            filtered ={key:value for key,value in matching_frame.items() if value==key }
            if len(matching_frame.keys())!=0:
                acc = acc + len(filtered.keys())/ len(matching_frame.keys())
                rec = rec+ len(filtered.keys())/ len(label_track[frame_id].keys())
                
                
                if len(filtered.keys())/ len(label_track[frame_id].keys())>1:
                    print("stop")
                #accs.append(acc)
                #recs.append(rec)
                accs.append(acc/nbr_frame)
                recs.append(rec/nbr_frame)
                if acc!=0.0 and recs!=0:
                    f1=2*(acc/nbr_frame)*(rec/nbr_frame)/((acc/nbr_frame)+(rec/nbr_frame))
                else:
                    f1=0.0
                    
                #f1s.append(f1)
                f1s.append(f1)
                indices.append(int(frame_id))


        else:
            if nbr_frame!=0:
                accs.append(acc/nbr_frame)
                recs.append(rec/nbr_frame)
                if acc!=0.0 and recs!=0:
                    f1=2*(acc/nbr_frame)*(rec/nbr_frame)/((acc/nbr_frame)+(rec/nbr_frame))
                else:
                    f1=0.0

                #f1s.append(f1)
                f1s.append(f1)
                indices.append(int(frame_id))
            
    acc = acc/nbr_frame
    rec=rec/nbr_frame
    f1=2*acc*rec/(acc+rec)
    print(acc, rec,f1) 
    return accs  , recs , f1s, indices            

Define global variable such as location of results

In [15]:
label_file= "/home/sophie/uncertain-identity-aware-tracking/Bytetrack/videos/labels_with_atq.json"
track_base = "/home/sophie/uncertain-identity-aware-tracking/Bytetrack/videos/results/"
label_track = read_data(label_file)


##plot accuracy, recall and f1-score over time

In [16]:

fig_acc = go.Figure()
fig_rec = go.Figure()
fig_f1 = go.Figure()
df =pd.DataFrame()
dct={"Bytetrack":"Bytetrack.json",  "hmm_with_atq":"GR77_20200512_111314_with_atq_tracking_with_HMM_result_feeder.json", "deepsort":"deepsort_smoothed_resut_without_max_thres.json","deeplabcut":"deeplabcut.json", "cross_input":"cross_input.json"}# 
for idx, method in enumerate(dct.keys()):
    track_result= read_data(track_base+dct[method]) #### a refaire
    
    if method == "hmm_with_atq":
        df[method+"accuracy"], df[method+"recall"], df[method+"f1-score"], indices = precise_accuracy_track(label_track, track_result, basic_tracker=False)
    else:
        df[method+"accuracy"], df[method+"recall"], df[method+"f1-score"], indices = precise_accuracy_track(label_track, track_result, basic_tracker=True)
        
    print(method)
    df.index =indices
    fig_rec.add_trace(go.Scatter(x=df.index.values, y=df[method+"recall"].values,
                            mode='lines',
                            name='recall of '+method))
    fig_f1.add_trace(go.Scatter(x=df.index.values, y=df[method+"f1-score"].values, 
                            mode='lines',
                            name='f1-score of '+method))
    fig_acc.add_trace(go.Scatter(x=df.index.values, y=df[method+"accuracy"].values,
                            mode='lines',
                            name='accuracy of '+method))
    
    
    df =pd.DataFrame()
        

fig_f1.update_layout(
            xaxis_title=dict(text="frame number"),
            yaxis_title=dict(text= "f1-score"),
    font=dict(
             #   color="black",
                size=22
            ),
            )
fig_acc.update_layout(
            xaxis_title="frame number",
            yaxis_title="accuracy",
    font=dict(
             #   color="black",
                size=22
            ),
            )
fig_rec.update_layout(
            xaxis_title="frame number",
            yaxis_title="recall",
    font=dict(
             #   color="black",
                size=22
            ),
            )

fig_f1.show()
fig_rec.show()
fig_acc.show()

{'13': '4817.0', '1': '4810.0', '6': '4811.0', '7': '4812.0', '11': '4807.0', '8': '4818.0', '5': '4815.0', '10': '4804.0', '15': '4808.0', '3': '4809.0'}
stop
0.6381697687535254 0.4428934010152213 0.522894845023361
Bytetrack
stop
0.7421895484157239 0.43519695044472545 0.5486705146272526
hmm_with_atq
{'13': '4817.0', '1': '4810.0', '6': '4811.0', '7': '4812.0', '11': '4807.0', '8': '4818.0', '5': '4815.0', '2': '4804.0', '15': '4808.0', '3': '4809.0'}
stop
0.4716645489199478 0.3319114176801597 0.3896354684768692
deepsort
{'5': '4817.0', '3': '4810.0', '6': '4811.0', '4': '4812.0', '12': '4807.0', '7': '4818.0', '1': '4804.0', '10': '4808.0', '2': '4809.0'}
stop
0.11928934010152427 0.0766860043509785 0.09335687486206144
deeplabcut
{'12': '4817.0', '0': '4810.0', '7': '4811.0', '3': '4812.0', '10': '4807.0', '11': '4818.0', '4': '4815.0', '2': '4804.0', '14': '4808.0', '1': '4809.0'}
0.1735735735735736 0.056949806949806954 0.08576120551555558
cross_input


Plot IDF1 and clear metric mota

In [17]:
dct={ "hmm_with_atq":"GR77_20200512_111314_with_atq_tracking_with_HMM_resut782_s.json"}#,"cross_input":"cross_input.json", "Bytetrack":"Bytetrack.json", "deepsort_smoothed_resut_without_max_thres":"deepsort_smoothed_resut_without_max_thres.json"}#, "deeplabcut":"deeplabcut_training.h5"}#, 
    #"deepsort":"deepsort_training.json", "deepsort_smoothed":"deepsort_training_smoothed.json", "deeplabcut":"deeplabcut_training.h5"}

dct={"Bytetrack":"Bytetrack.json",  "hmm_with_atq":"GR77_20200512_111314_with_atq_tracking_with_HMM_result_feeder.json", "deepsort":"deepsort_smoothed_resut_without_max_thres.json","deeplabcut":"deeplabcut.json", "cross_input":"cross_input.json"}# 

for key, value in dct.items():
    if ".h5" not in value:
        model_track = read_data(track_base+value)   
    else:
        model_track = deeplabcut_model_track
    model_name=key
    
    
    model_summary=Evaluate_over_time(label_track, model_track, model_name)
    model_summary.to_excel(model_name+"_Tracking_result.xlsx",index_label="index")

    print("#################################################################DF")

    #model_summary_sf=Evaluate_single_frames_over_time(tracks, model_track, model_name)
    #model_summary_sf.to_excel("sf_"+model_name+"_Tracking_result.xlsx",index_label="index")


cumulative summary
         IDF1   IDP   IDR  Rcll  Prcn   GT   MT  PT ML     FP    FN IDs     FM  MOTA  MOTP IDt IDa IDm
109     87.9% 85.0% 91.1% 91.1% 85.0%   14   11   3  0     18    10   0      2 75.0% 0.486   0   0   0
218     82.8% 80.0% 85.7% 85.7% 80.0%   14   11   3  0     39    26   0      7 64.3% 0.488   0   0   0
396     82.8% 80.0% 85.7% 85.7% 80.0%   14   11   3  0     57    38   0     15 64.3% 0.486   0   0   0
508     83.7% 80.9% 86.7% 86.7% 80.9%   14   11   3  0     63    41   0     16 66.2% 0.483   0   0   0
621     84.4% 81.6% 87.4% 87.4% 81.6%   14   11   3  0     80    51   0     22 67.7% 0.470   0   0   0
734     85.7% 82.8% 88.7% 88.7% 82.8%   14   11   3  0     85    52   0     25 70.3% 0.467   0   0   0
844     85.9% 83.1% 89.0% 89.0% 83.1%   14   11   3  0     94    57   0     27 70.8% 0.468   0   0   0
944     85.4% 82.5% 88.4% 88.4% 82.5%   14   11   3  0    110    68   0     29 69.7% 0.470   0   0   0
1061    84.6% 81.7% 87.6% 87.6% 81.7%   14   11   3  0

cumulative summary
         IDF1   IDP   IDR  Rcll  Prcn   GT  MT  PT  ML    FP     FN  IDs     FM  MOTA  MOTP IDt IDa IDm
109     59.7% 82.1% 46.9% 46.9% 82.1%   14   6   1   7    10     52    0      0 36.7% 0.437   0   0   0
218     56.1% 77.1% 44.0% 44.0% 77.1%   14   6   1   7    22     94    0      3 31.0% 0.438   0   0   0
396     55.6% 76.4% 43.7% 43.7% 76.4%   14   6   1   7    34    142    0      8 30.2% 0.461   0   0   0
508     55.8% 76.8% 43.9% 43.9% 76.8%   14   6   1   7    39    165    0      9 30.6% 0.460   0   0   0
621     57.1% 78.6% 44.9% 44.9% 78.6%   14   6   1   7    48    216    0     11 32.7% 0.445   0   0   0
734     58.0% 79.7% 45.5% 45.5% 79.7%   14   6   1   7    52    244    0     12 33.9% 0.439   0   0   0
844     58.1% 79.9% 45.6% 45.6% 79.9%   14   6   1   7    58    274    0     13 34.1% 0.442   0   0   0
944     57.9% 79.6% 45.5% 45.5% 79.6%   14   6   1   7    67    313    0     13 33.8% 0.441   0   0   0
1061    57.7% 79.3% 45.3% 45.3% 79.3%   14   

cumulative summary
         IDF1   IDP   IDR  Rcll  Prcn   GT   MT  PT ML     FP    FN  IDs     FM  MOTA  MOTP  IDt IDa IDm
109     87.7% 84.8% 90.8% 90.8% 84.8%   14   11   3  0     16     9    0      0 74.5% 0.493    0   0   0
218     82.2% 79.4% 85.1% 85.1% 79.4%   14   11   3  0     37    25    0      5 63.1% 0.492    0   0   0
396     82.4% 79.4% 85.7% 86.5% 80.1%   14   11   3  0     54    34    2     13 64.3% 0.485    0   1   0
508     83.5% 80.4% 86.7% 87.4% 81.1%   14   11   3  0     60    37    2     14 66.3% 0.481    0   1   0
621     85.0% 82.0% 88.3% 88.8% 82.5%   14   11   3  0     74    44    2     18 69.4% 0.469    0   1   0
734     86.2% 83.2% 89.5% 90.0% 83.6%   14   11   3  0     79    45    2     20 71.9% 0.466    0   1   0
844     86.4% 83.4% 89.7% 90.1% 83.8%   14   11   3  0     88    50    2     22 72.2% 0.466    0   1   0
944     86.1% 83.1% 89.4% 89.7% 83.5%   14   12   2  0    102    59    2     24 71.6% 0.468    0   1   0
1061    85.5% 82.5% 88.6% 88.9% 82.8

cumulative summary
         IDF1   IDP   IDR  Rcll  Prcn   GT  MT   PT ML     FP     FN   IDs     FM  MOTA  MOTP   IDt IDa IDm
109     78.4% 75.8% 81.2% 83.9% 78.3%   14  10    3  1     26     18     2      5 58.9% 0.361     0   1   0
218     77.5% 74.9% 80.2% 81.9% 76.4%   14  11    2  1     46     33     2     12 55.5% 0.386     0   1   0
396     77.7% 75.1% 80.5% 82.7% 77.2%   14  12    1  1     65     46     5     16 56.4% 0.421     3   1   0
508     74.9% 72.4% 77.6% 81.8% 76.4%   14  11    2  1     78     56    10     20 53.2% 0.427     7   1   0
621     68.0% 65.7% 70.4% 79.8% 74.5%   14   9    4  1    111     82    13     25 49.3% 0.433    11   1   1
734     64.2% 62.0% 66.5% 79.2% 73.9%   14  10    3  1    129     96    16     31 47.8% 0.434    14   1   1
844     61.7% 59.6% 63.9% 79.3% 74.1%   14  10    4  0    144    107    18     36 48.1% 0.439    16   1   1
944     58.8% 56.8% 60.9% 78.7% 73.5%   14  10    4  0    167    125    22     40 46.6% 0.443    19   1   1
1061    5

cumulative summary
         IDF1   IDP   IDR  Rcll  Prcn   GT   MT  PT ML    FP    FN   IDs     FM  MOTA  MOTP IDt   IDa IDm
109     80.5% 77.8% 83.3% 90.5% 84.4%   14   11   3  0     7     4     3      0 66.7% 0.474   0     3   0
218     67.8% 65.6% 70.2% 83.3% 77.8%   14   11   2  1    20    14     6      2 52.4% 0.467   0     6   0
396     70.0% 67.5% 72.7% 85.1% 78.9%   14   10   4  0    35    23    10      9 55.8% 0.469   0    10   0
508     69.0% 66.7% 71.4% 86.2% 80.5%   14   10   4  0    41    27    12     11 59.2% 0.466   0    12   0
621     67.6% 65.3% 70.0% 86.2% 80.4%   14   10   4  0    44    29    15     12 58.1% 0.461   0    15   0
734     65.1% 62.9% 67.4% 87.1% 81.2%   14   10   4  0    45    29    21     14 57.6% 0.462   0    21   0
844     59.5% 57.5% 61.7% 88.0% 82.1%   14   10   4  0    51    32    25     15 59.4% 0.463   0    25   0
944     54.9% 53.0% 56.8% 87.7% 81.8%   14   11   3  0    60    38    30     17 58.4% 0.469   0    30   0
1061    53.1% 51.3% 55.0% 8

In [18]:
#dct={"Bytetrack":"Bytetrack.json", "cross_input":"cross_input.json", "deepsort_smoothed_resut_without_max_thres":"deepsort_smoothed_resut_without_max_thres.json","deeplabcut":"deeplabcut_training.h5"}# 
dct={"Bytetrack":"Bytetrack.json",  "hmm_with_atq":"GR77_20200512_111314_with_atq_tracking_with_HMM_result_feeder.json", "deepsort":"deepsort_smoothed_resut_without_max_thres.json","deeplabcut":"deeplabcut.json", "cross_input":"cross_input.json"}# 

#"hmm_with_atq":"GR77_20200512_111314_with_atq_tracking_with_HMM_resut782.json"#
import pandas as pd
import copy
for idx, method in enumerate(dct.keys()):
    df_tmp = pd.read_excel(method+"_Tracking_result.xlsx")
    df_tmp.index = df_tmp['index']
    df_tmp = df_tmp.drop(['index'], axis=1)

    if idx>0:
        df_final = pd.merge(df_final, df_tmp, left_index=True, right_index=True)
    else:
        print(df_tmp)
        df_first =copy.deepcopy(df_tmp)
        df_final =df_tmp
    
"""df_deepsort=pd.read_excel("deepsort_Tracking_result.xlsx")
df_deepsort.index = df_deepsort['index']

df_Bytetrack=pd.read_excel("Bytetrack_Tracking_result.xlsx")
df_Bytetrack.index = df_Bytetrack['index']

df_deeplabcut = pd.read_excel("deeplabcut_Tracking_result.xlsx")
df_deeplabcut.index = df_deeplabcut['index']

df_deepsort_smoothed = pd.read_excel("deepsort_smoothed_Tracking_result.xlsx")
df_deepsort_smoothed.index = df_deepsort_smoothed['index']

df_deepsort_smoothed_without_max =  pd.read_excel("deepsort_smoothed_resut_without_max_thres_Tracking_result.xlsx")
df_deepsort_smoothed_without_max.index =  df_deepsort_smoothed_without_max["index"]

df_cross_input =  pd.read_excel("cross_input_Tracking_result.xlsx")
df_cross_input.index =  df_cross_input["index"]

df_hmm=  pd.read_excel("hmm_with_atq_Tracking_result.xlsx")
df_hmm.index =  df_hmm["index"]



df = pd.merge(df_deepsort, df_deeplabcut, left_index=True, right_index=True)
df2 = pd.merge(df, df_deepsort_smoothed, left_index=True, right_index=True)
df3 = pd.merge(df2, df_deepsort_smoothed_without_max, left_index=True, right_index=True)
df4= pd.merge(df3, df_Bytetrack, left_index=True, right_index=True)
df5= pd.merge(df4, df_cross_input, left_index=True, right_index=True)
df6= pd.merge(df5, df_hmm, left_index=True, right_index=True)

df=df6
print(df.columns)
"""
import plotly.express as px
import plotly.graph_objects as go

df=  df_final
print(df.columns)
for col in df_first:# ["num_switches","mota", "idf1"]:# df_first: #['idp', 'mota','idr', idf1 ]:#
    col =col.split('_Bytetrack')[0]
    if "index" not in col:
        fig = go.Figure()
        print(col)
        """fig.add_trace(go.Scatter(x=df.index.values, y=df[col+"_deepsort"].values,
                            mode='lines',
                            name=col+' of deepsort'))

        fig.add_trace(go.Scatter(x=df.index.values, y=df[col+"_deepsort_smoothed"].values,
                                mode='lines',
                                name=col+' of deepsort_smoothed'))"""

        fig.add_trace(go.Scatter(x=df.index.values, y=df[col+"_deeplabcut"].values,
                            mode='lines',
                            name=col+' of deeplabcut'))
        
        fig.add_trace(go.Scatter(x=df.index.values, y=df[col+"_deepsort"].values,
                            mode='lines',
                            name=col+' of deepsort'))
        fig.add_trace(go.Scatter(x=df.index.values, y=df[col+"_Bytetrack"].values,
                            mode='lines',
                            name=col+' of Bytetrack'))
        fig.add_trace(go.Scatter(x=df.index.values, y=df[col+"_cross_input"].values,
                            mode='lines',
                            name=col+' of cross_input'))
        fig.add_trace(go.Scatter(x=df.index.values, y=df[col+"_hmm_with_atq"].values,
                            mode='lines',
                            name=col+' of hmm_with_atq'))
        fig.update_layout(
            xaxis_title="frame number",
            yaxis_title=col,
            
            
    font=dict(
             #   color="black",
                size=22
            ),
                      
            )
        fig.show() 

         idf1_Bytetrack  idp_Bytetrack  idr_Bytetrack  recall_Bytetrack  \
index                                                                     
109            0.879310       0.850000       0.910714          0.910714   
218            0.827586       0.800000       0.857143          0.857143   
396            0.827586       0.800000       0.857143          0.857143   
508            0.836991       0.809091       0.866883          0.866883   
621            0.844233       0.816092       0.874384          0.874384   
...                 ...            ...            ...               ...   
14456          0.780078       0.762966       0.797975          0.856957   
14562          0.779318       0.762370       0.797036          0.856038   
14670          0.778036       0.761260       0.795567          0.855136   
14777          0.777729       0.761062       0.795141          0.855058   
OVERALL        0.774402       0.751873       0.798323          0.867139   

         precision_Bytet

idp


idr


recall


precision


...

mota


motp


num_transfer


num_ascend


num_migrate
